# A Monte Carlo program for the Newman-Moore triangular lattice Ising model

In [1]:
using Random; Random.seed!(0); 
rng = MersenneTwister(1234);
#k = 2
#L = 2^k
L = 100
N = L*L
Spin = fill(1,N);

In [2]:
#This is the data structure that relates up-triangles to spin indices
UpTriangle = zeros(Int,N,3)
for i = 1:N
    UpTriangle[i,1] = i
    UpTriangle[i,2] = i+1    
    UpTriangle[i,3] = i+L
    #fix PBCs
    if mod(i,L) == 0
        UpTriangle[i,2] = i + 1 - L
    end
    if (i+L)> N
        UpTriangle[i,3] = i + L - N
    end
end

In [3]:
#This is the inverse data structure that relates a spin index to its 3 up-triangles
AssociatedTri = zeros(Int,N,3)
for i = 1:N
    AssociatedTri[i,1] = i
    AssociatedTri[i,2] = i-1
    AssociatedTri[i,3] = i-L
    if mod(i-1+L,L) == 0
        AssociatedTri[i,2] = i - 1 + L
    end
    if i<(L+1) 
        AssociatedTri[i,3] = i + N - L
    end
end

In [4]:
#here is the brute force calculation of the energy
function Energy_Total(Energy,Spin)
    Energy = 0
    for i = 1:N
        Spin1 = Spin[UpTriangle[i,1]]
        Spin2 = Spin[UpTriangle[i,2]]
        Spin3 = Spin[UpTriangle[i,3]]
        Energy += 0.5 * Spin1 * Spin2 * Spin3  #J = 1
    end
    return Energy
end #Energy_Total

Energy_Total (generic function with 1 method)

In [5]:
#here is the energy DIFFERENCE calculated from the local triangular plaquettes
function Energy_Diff(Spin,spin_index)
    
    SpinCopy = copy(Spin)
    
    Tri1 = AssociatedTri[spin_index,1]
    Tri2 = AssociatedTri[spin_index,2]
    Tri3 = AssociatedTri[spin_index,3]

    local_e_before = 0
    Spin1 = SpinCopy[UpTriangle[Tri1,1]]
    Spin2 = SpinCopy[UpTriangle[Tri1,2]]
    Spin3 = SpinCopy[UpTriangle[Tri1,3]]
    local_e_before += 0.5 * Spin1 * Spin2 * Spin3  
    Spin1 = SpinCopy[UpTriangle[Tri2,1]]
    Spin2 = SpinCopy[UpTriangle[Tri2,2]]
    Spin3 = SpinCopy[UpTriangle[Tri2,3]]
    local_e_before += 0.5 * Spin1 * Spin2 * Spin3  
    Spin1 = SpinCopy[UpTriangle[Tri3,1]]
    Spin2 = SpinCopy[UpTriangle[Tri3,2]]
    Spin3 = SpinCopy[UpTriangle[Tri3,3]]
    local_e_before += 0.5 * Spin1 * Spin2 * Spin3  

    #flip the spin
    SpinCopy[spin_index] *= -1
    
    local_e_after = 0
    Spin1 = SpinCopy[UpTriangle[Tri1,1]]
    Spin2 = SpinCopy[UpTriangle[Tri1,2]]
    Spin3 = SpinCopy[UpTriangle[Tri1,3]]
    local_e_after += 0.5 * Spin1 * Spin2 * Spin3  
    Spin1 = SpinCopy[UpTriangle[Tri2,1]]
    Spin2 = SpinCopy[UpTriangle[Tri2,2]]
    Spin3 = SpinCopy[UpTriangle[Tri2,3]]
    local_e_after += 0.5 * Spin1 * Spin2 * Spin3  
    Spin1 = SpinCopy[UpTriangle[Tri3,1]]
    Spin2 = SpinCopy[UpTriangle[Tri3,2]]
    Spin3 = SpinCopy[UpTriangle[Tri3,3]]
    local_e_after += 0.5 * Spin1 * Spin2 * Spin3  
    
    #println(spin_index," ",local_e_before," ",local_e_after)
    
    return local_e_after - local_e_before
end

Energy_Diff (generic function with 1 method)

In [6]:
T = 1.0
beta = 1.0/T

#initialize the energy
Energy = 0
Energy=Energy_Total(Energy,Spin)
#println(Energy)

for step = 1:1000
    
    spin_i = rand(1:N)
    DeltaE = Energy_Diff(Spin,spin_i)
    
    #Metropolis update
    if DeltaE <= 0
        Energy += DeltaE
        Spin[spin_i] *= -1
    else
        rnum = rand(rng)  #random number for Metropolis
        #println(rnum," ",DeltaE," ",exp(-beta*DeltaE))
        if (exp(-beta*DeltaE) > rnum)
            Energy += DeltaE
            Spin[spin_i] *= -1
        end
    end #Metropolis
        
end #step

println(Energy)


2656.0


In [7]:
Energy=Energy_Total(Energy,Spin)
println(Energy)

2656.0
